https://keras.io/examples/nlp/text_classification_from_scratch/

In [1]:
import pandas as pd
from collections import Counter
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# TF tensors

In [2]:
# tf.constant creates a tensor
tf_tensor = tf.constant(["Привет", "Как дела"])

In [3]:
tf_tensor.shape

TensorShape([2])

In [4]:
# NB: Russian lang is represented as a byte strings -> that won't cause any problems
tf_tensor

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'\xd0\x9f\xd1\x80\xd0\xb8\xd0\xb2\xd0\xb5\xd1\x82',
       b'\xd0\x9a\xd0\xb0\xd0\xba \xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb0'],
      dtype=object)>

# Load csv data 

In [5]:
ls ../data/raw

lenta_10k_3_classes.csv  readme.md


In [6]:
path = '../data/raw/lenta_10k_3_classes.csv'

In [7]:
df = pd.read_csv(path, encoding = 'utf8')

In [8]:
# shuffle rows 
df = df.sample(frac = 1)

In [9]:
# df = df[:1000]

In [10]:
df.head()

,text,topic
1142,Цены на нефть за прошедшую неделю по итогам то...,Экономика
5115,В России будут созданы два мощных автомобильны...,Экономика
8872,"Президент Владимир Путин, выступая на проходящ...",Экономика
9054,Министерство России по антимонопольной политик...,Экономика
6992,В колоннаде Казанского собора Санкт-Петербурга...,Культура


In [11]:
# Take a glance at unique classes
df['topic'].unique()

array(['Экономика', 'Культура', 'Спорт'], dtype=object)

In [12]:
# Distributions
Counter(df['topic'])

Counter({'Экономика': 4902, 'Культура': 2928, 'Спорт': 2170})

In [13]:
def create_dir_path(*dir_names):
    """
    Concatenates variable num of dir names using forward slash
    """
    
    path = ''
    for name in dir_names:
        path += "/" + name
    
    return path[1:]

In [14]:
def create_text_files_dir(df, text_col='text', label_col='topic', dir_root_name='dataset', path='../data/interim'):
    """
    Creates a text dir w/ the following structure from a dataframe:
     main_directory/  
    ...class_a/  
    ......a_text_1.txt  
    ......a_text_2.txt  
    ...class_b/  
    ......b_text_1.txt  
    ......b_text_2.txt
    """
    
    # create dirs with label names if they don't exhist yet
    class_names = df['topic'].unique()
    for class_name in class_names:
        class_name_dir_path = create_dir_path(path, dir_root_name, class_name)
        
        if not os.path.exists(class_name_dir_path):
            os.makedirs(class_name_dir_path)
        
        # fill the dirs w/ text files
        class_df = df[df['topic']==class_name]
        for i, row in class_df.iterrows():
            with open(f'{class_name_dir_path}/{class_name}_text_{i}.txt', 'w+') as f:
                f.write(row['text'])

In [15]:
# create train and test dfs
train_df, test_df = train_test_split(df, test_size=0.2)

In [16]:
train_df.shape

(8000, 2)

In [17]:
test_df.shape

(2000, 2)

In [18]:
# creating directories
create_text_files_dir(train_df, dir_root_name='train')
create_text_files_dir(test_df, dir_root_name='test')

In [19]:
label_names = list(os.walk('../data/interim/test'))[0][1]
label_names

['Культура', 'Экономика', 'Спорт']

# Create train, val, test tf Datasets

In [20]:
# NB subset param, it enables taking diff splits -> seed must be the same
# label_mode='categorical' for categorical_crossentropy loss

In [21]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/interim/train",
    label_mode='int',
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
    class_names=label_names
)

Found 9585 files belonging to 3 classes.
Using 7668 files for training.


In [22]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/interim/train",
    label_mode='int',
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    class_names=label_names
    
)

Found 9585 files belonging to 3 classes.
Using 1917 files for validation.


In [23]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/interim/test",
    batch_size=batch_size, 
    label_mode='int',
    class_names=label_names
)

Found 3585 files belonging to 3 classes.


In [24]:
print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)

Number of batches in raw_train_ds: 240
Number of batches in raw_val_ds: 60
Number of batches in raw_test_ds: 113


In [25]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(6):
        print(label_batch.numpy()[i])

1
0
2
2
1
2


# Vectorization

In [26]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [27]:
max_features = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [28]:
vocab = vectorize_layer.get_vocabulary()

In [29]:
len(vocab)

0

In [30]:
'акций' in vocab

False

In [31]:
vocab

[]

In [32]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [33]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [34]:
vectorize_text('Привет как дела', 1)

(<tf.Tensor: shape=(1, 100), dtype=int64, numpy=
 array([[  1,  23, 917,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0]])>,
 1)

In [35]:
vectorize_layer.get_vocabulary()[707]

'1996'

In [36]:
# Apply verctorization to dataset like a function

In [37]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [38]:
# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

# Model

In [39]:
embedding_dim = 300

In [40]:
# A integer input for vocab indices.
# inputs = tf.keras.Input(shape=(None,), dtype="int64")
inputs = tf.keras.Input(shape=(100,), dtype="int64")


# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.LSTM(64)(x)
predictions = layers.Dense(3, activation="sigmoid", name="predictions")(x)

In [41]:
model = tf.keras.Model(inputs, predictions)
# opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
opt = 'Adam'

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [42]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
240/240 [==============================] - 16s 69ms/step - loss: 0.6399 - accuracy: 0.7297 - val_loss: 0.7633 - val_accuracy: 0.7235
Epoch 2/10
240/240 [==============================] - 17s 69ms/step - loss: 0.4439 - accuracy: 0.8679 - val_loss: 0.3202 - val_accuracy: 0.9317
Epoch 3/10
240/240 [==============================] - 17s 71ms/step - loss: 0.2147 - accuracy: 0.9463 - val_loss: 0.1946 - val_accuracy: 0.9572
Epoch 4/10
240/240 [==============================] - 18s 75ms/step - loss: 0.1275 - accuracy: 0.9656 - val_loss: 0.1183 - val_accuracy: 0.9661
Epoch 5/10
240/240 [==============================] - 18s 74ms/step - loss: 0.0704 - accuracy: 0.9834 - val_loss: 0.1052 - val_accuracy: 0.9760
Epoch 6/10
240/240 [==============================] - 16s 67ms/step - loss: 0.0407 - accuracy: 0.9920 - val_loss: 0.0936 - val_accuracy: 0.9760
Epoch 7/10
240/240 [==============================] - 16s 66ms/step - loss: 0.0225 - accuracy: 0.9958 - val_loss: 0.0804 - val_accuracy:

In [43]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

113/113 [==============================] - 1s 12ms/step - loss: 0.0457 - accuracy: 0.9919
Loss:  0.04570648446679115
Accuracy:  0.9919107556343079


# End to end model

In [44]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

113/113 [==============================] - 2s 14ms/step - loss: 0.0457 - accuracy: 0.9919


[0.04570648819208145, 0.9919107556343079]

In [45]:
import numpy as np

In [46]:
def predict(text):
    pred_probas = end_to_end_model.predict([text])
    idx = np.argmax(pred_probas)
    
    return label_names[idx]

# Sanity check

In [47]:
for i, row in df[:20].iterrows():
    print('Text: ', row['text'])
    print('True: ', row['topic'])
    print('Pred: ', predict(row['text']))
    print()

Text:  Цены на нефть за прошедшую неделю по итогам торгов на биржах в Лондоне и Нью-Йорке снизились в среднем на 7,9 процента и достигли рекордно низкого за последние пять месяцев уровня, сообщает РБК. По итогам торгов в пятницу 7 апреля цены фьючерсных контрактов со сроком поставки в мае составили 22,58 доллара за баррель нефти (марка Brent, Лондонская нефтяная биржа) и 25,04 доллара за баррель (марка Light Sweet, Нью-Йоркская биржа). Одновременно началось падение акций ведущих мировых нефтяных компаний, составляющих основу расчета индекса S&P International Oil Index. Падение цен на нефть вызвано целым рядом факторов, главным из которых остается возросшие с 1 апреля поставки нефти 9 странами ОПЕК. Стимулирует снижение цен на нефть информация о росте запасов сырой нефти и бензина в США, а также возможность дополнительного увеличения поставок нефти странами ОПЕК в третьем квартале. Между тем в пятницу один из крупнейших поставщиков нефти на мировой рынок Норвегия заявила, что в мае пере

Pred:  Спорт

Text:  Лишь два российских теннисиста - Евгений Кафельников и Марат Сафин - принимают участие в Уимблдонском турнире. Свои первые матчи оба проводили во вторник. И добились относительно легких побед. Кафельников встречался с молодым швейцарцем Роджером Федерером. В каждом сете равная борьба велась до решающих геймов. Однако, в концовке лучше неизменно действовал россиянин, посеянный на турнире под 5-м номером. Сочинец добился победы в трех партиях - 7:5, 7:5, 7:6. Сафина жребий в первом же круге свел с испанцем Гало Бланко. Теннисист с Пириней, не слишком удачно выступающий на быстрых покрытиях, лишь в первом сете сумел завязать более-менее равную борьбу. В дальнейшем преимущество Сафина было бесспорным - 7:6, 6:3, 6:4. Во вторник свой первый матч на нынешнем Уимблдоне выиграл посеянный под вторым номером Андре Агасси, переигравший своего соотечественника Тейлора Дента. Вышли во второй круг и другие известные теннисисты - Тим Хенмен, Густаво Куэртен, Магнус Норман, Марк Ф

In [48]:
predict('На счета Минфина сегодня поступили средства японского государственного')

'Экономика'

In [49]:
predict(' понедельник стал известен состав лыжной сборной России, которая выступит на чемпионате мира в финском Лахти')

'Спорт'